In [2]:
import EarthModel
import numpy as np
import math
import sys
sys.path.append('C:\\Users\\stone\\source\\nav')
import quaternion
import RotationConversion

In [3]:
testEuler1 = {
    'NAV_FRAME_EULER_ROLL':0,
    'NAV_FRAME_EULER_PITCH':90*np.pi/180.0,
    'NAV_FRAME_EULER_HEADING':0.0
}

In [4]:
print(testEuler1)

{'NAV_FRAME_EULER_ROLL': 0, 'NAV_FRAME_EULER_PITCH': 1.5707963267948966, 'NAV_FRAME_EULER_HEADING': 0.0}


In [5]:
testQ1 = RotationConversion.FormQuaternionFromEulerAngles(testEuler1)

In [6]:
print(testQ1)

{'s': np.float64(0.7071067811865476), 'i': np.float64(0.0), 'j': np.float64(0.7071067811865476), 'k': np.float64(0.0)}


In [7]:
testQ2 = { 's':1,'i':2,'j':3,'k':4}
testQ = { 's':0.707106781,'i':0,'j':0,'k':0.707106781}
q1 = quaternion.Quaternion_RotationIdentity()
q2 = quaternion.sQuaternion_Copy(q1)
q3 = quaternion.Quaternion_Multiply(testQ,q1)
q4 = quaternion.Quaternion_Normalize(testQ2)

In [8]:
print(q1)
print(q2)
print(q3)
print(q4)

{'s': 1.0, 'i': 0.0, 'j': 0.0, 'k': 0.0}
{'s': 1.0, 'i': 0.0, 'j': 0.0, 'k': 0.0}
{'s': 0.707106781, 'i': 0.0, 'j': 0.0, 'k': 0.707106781}
{'s': 0.18257418583505536, 'i': 0.3651483716701107, 'j': 0.5477225575051661, 'k': 0.7302967433402214}


In [9]:
dcm = RotationConversion.FormDCMFromQuaternion(testQ)
euler = RotationConversion.ExtractEulerAnglesFromDCM(dcm)
testEuler = {'NAV_FRAME_EULER_ROLL':0.0,
             'NAV_FRAME_EULER_PITCH':90.0*np.pi/180.0,
             'NAV_FRAME_EULER_HEADING':0.0
             }
qq = RotationConversion.FormQuaternionFromEulerAngles(testEuler)

In [10]:
print(dcm)
print(euler)
print(qq)

[[ 0. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]
{'NAV_FRAME_EULER_ROLL': np.float64(0.0), 'NAV_FRAME_EULER_PITCH': np.float64(-0.0), 'NAV_FRAME_EULER_HEADING': np.float64(1.5707963267948966)}
{'s': np.float64(0.7071067811865476), 'i': np.float64(0.0), 'j': np.float64(0.7071067811865476), 'k': np.float64(0.0)}
